# Install Necessary libraries

In [107]:
!pip install gradio
!pip install nltk
!pip install pandas
!pip install rapidfuzz
!pip install nltk fuzzywuzzy

In [108]:
import gradio as gr
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Load the pre-trained model for image recognition(Resnet)

In [109]:
model = ResNet50(weights='imagenet')

# Load and preprocess the food dataset

In [110]:
data = pd.read_excel('/content/Theta_bot_updated.xlsx')
data = data[['restaurant_id', 'category', 'name', 'description', 'price', 'Review']]
data.drop_duplicates(inplace=True)
data.dropna(subset=['name', 'price', 'Review'], inplace=True)


In [111]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
data['sentiment_score'] = data['Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])
data['sentiment_category'] = data['sentiment_score'].apply(lambda x: 'Very Good' if x >= 0.6 else ('Good' if x >= 0.3 else 'Average'))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [112]:
food_sentiment = data.groupby(['restaurant_id', 'name']).agg({'sentiment_score': 'mean', 'sentiment_category': lambda x: pd.Series.mode(x)[0]}).reset_index()
restaurant_details = data[['restaurant_id', 'category', 'name', 'description', 'price']].drop_duplicates()
food_sentiment = pd.merge(food_sentiment, restaurant_details, on=['restaurant_id', 'name'])

In [113]:
sentiment_order = {'Very Good': 1, 'Good': 2, 'Average': 3}

In [114]:
def search_food(food_item):
    filtered_foods = food_sentiment[food_sentiment['name'].str.contains(food_item, case=False, na=False)].copy()
    filtered_foods['sentiment_rank'] = filtered_foods['sentiment_category'].map(sentiment_order)

# Improved price cleaning and conversion
    filtered_foods['price'] = (
        filtered_foods['price']
        .astype(str)
        .str.replace('[^\\d.]', '', regex=True)
    )
    filtered_foods['price'] = pd.to_numeric(filtered_foods['price'], errors='coerce')
    filtered_foods.dropna(subset=['price'], inplace=True)

    sorted_foods = filtered_foods.sort_values(by=['sentiment_rank', 'price'], ascending=[True, True])

    if sorted_foods.empty:
        return f'No restaurants found for "{food_item}".'

    result = ''
    for _, row in sorted_foods.iterrows():
        result += 'Food Item: {}, Restaurant ID: {}, Review: {}, Price: $ {:.2f} USD\n'.format(
            row['name'], row['restaurant_id'], row['sentiment_category'], row['price']
        )

    return result

# Function to recognize food in an image and perform sentiment analysis

In [115]:
def recognize_and_analyze_food(img):
#Recognize the food item in the image
 img = img.resize((224, 224))
 img_array = image.img_to_array(img)
 img_array = np.expand_dims(img_array, axis=0)
 img_array = preprocess_input(img_array)

 predictions = model.predict(img_array)
 decoded_predictions = decode_predictions(predictions, top=3)[0]
 top_food_label = decoded_predictions[0][1]

 # Check if the food item exists in the dataset
 result = search_food(top_food_label)
 return f"Recognized Food: {top_food_label}\n\n{result}"


#Deploy In Gradio

In [116]:
# Create Gradio interface with multiple tabs
with gr.Blocks() as demo:
    with gr.Tab("show me the food"):
        with gr.Row():
            image_input = gr.Image(type="pil", label="Upload Image of Food")
            image_output = gr.Textbox(label="Here's What I Found")
        with gr.Row():
            clear_image_button = gr.Button("Clear")

        def clear_image():
            return None, ""

        image_input.change(recognize_and_analyze_food, inputs=image_input, outputs=image_output)
        clear_image_button.click(clear_image, outputs=[image_input, image_output])

    with gr.Tab("Tell Us What You Want"):
        with gr.Row():
            food_input = gr.Textbox(label='Enter a food item')
            food_output = gr.Textbox(label='Available Options')
        with gr.Row():
            clear_food_button = gr.Button("Clear")

        def clear_food():
            return "", ""

        food_input.change(search_food, inputs=food_input, outputs=food_output)
        clear_food_button.click(clear_food, outputs=[food_input, food_output])

# Launch the Gradio app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://138f4bfea6f58b4265.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
